### transform the target variables from CSV to netcdf, and saves to disk 

In [ ]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline

### os 
import os 
import sys

### datetimes 
from datetime import datetime, timedelta

### scipy 
import numpy as np 
import pandas as pd
import xarray as xr

### plotting 
from matplotlib import pyplot as plt
import cartopy.crs as ccrs

In [3]:
import pathlib

HOME = pathlib.Path.home()
CWD = pathlib.Path.cwd() 

### paths where the CSV files with the target variables (time-series of seasonal anomalies and categories) reside

In [4]:
csv_paths = HOME.joinpath('research/Smart_Ideas/outputs/targets/NZ_regions/NZ_6_regions')

### list of variables 

In [5]:
list_vars = ['RAIN','TMEAN','SOILM'] 

### list of regions

In [6]:
list_regions = ['NNI','WNI','ENI','NSI','WSI','ESI']

In [7]:
for var_name in list_vars: 
    
    # get the terciles 
    num_quantiles = 3
    
    dset_terciles = []
    
    for region_name in list_regions: 
        lfiles = list(csv_paths.joinpath(f"{var_name}/{region_name}").glob(f"TS_NZ_region_*_{num_quantiles}_quantiles_*_regionmask.csv")) 
        data = pd.read_csv(lfiles[0], index_col=0, parse_dates=True)
        data = data.to_xarray()
        dset_terciles.append(data)
    dset_terciles = xr.concat(dset_terciles, pd.Index(list_regions, name='region'))

    # get the quintiles
    num_quantiles = 5
    
    dset_quintiles = []
    
    for region_name in list_regions: 
        lfiles = list(csv_paths.joinpath(f"{var_name}/{region_name}").glob(f"TS_NZ_region_*_{num_quantiles}_quantiles_*_regionmask.csv")) 
        data = pd.read_csv(lfiles[0], index_col=0, parse_dates=True)
        data = data.to_xarray()
        dset_quintiles.append(data)
        
    dset_quintiles = xr.concat(dset_quintiles, pd.Index(list_regions, name='region'))
    
    # merge 
    dset_target = xr.merge([dset_terciles, dset_quintiles['cat_5']])
    
    # saves to disk (several locations)
    
    # local drive 
    dset_target.to_netcdf(HOME.joinpath(f"research/Smart_Ideas/outputs/targets/NZ_regions/NZ_6_regions/{var_name}/NZ_6_regions_{var_name}_terciles_and_quintiles.nc")) 

    # media 
    dpath = pathlib.Path('/media/nicolasf/END19101/outputs/targets/NZ_regions/NZ_6_regions') 
    
    dset_target.to_netcdf(dpath.joinpath(f"{var_name}/NZ_6_regions_{var_name}_terciles_and_quintiles.nc"))  